In [18]:
import os
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]
account_id_hedge = os.environ["ACCOUNT_HEDGE"]
import oandapyV20
import oandapyV20.endpoints.transactions as trans
import pandas as pd

In [77]:
client = oandapyV20.API(access_token=access_token)
param = { "id": 900 }
r = trans.TransactionsSinceID(accountID=account_id_demo,  params=param)
trades= client.request(r)
param1 = { "id": 0 }
request_hedge = trans.TransactionsSinceID(accountID=account_id_hedge,  params=param1)
trades_hedge= client.request(request_hedge)

In [78]:
all_transactions_hedge = pd.DataFrame(trades_hedge['transactions'])
all_transactions = pd.DataFrame(trades['transactions'])
all_transactions = pd.concat([all_transactions_hedge,all_transactions])
all_transactions['time'] = pd.to_datetime(all_transactions['time'])
all_transactions['date'] = all_transactions['time'].dt.date
all_transactions = all_transactions[all_transactions['date'] == all_transactions['date'].max()]
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
print(all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby(['accountID','reason'])['pl'].sum())
print(all_transactions['pl'].sum())

accountID             reason                  
101-004-31059296-001  TAKE_PROFIT_ORDER           2353.7937
                      TRAILING_STOP_LOSS_ORDER   -5911.3750
101-004-31059296-003  TAKE_PROFIT_ORDER           3757.7192
                      TRAILING_STOP_LOSS_ORDER   -4347.5184
Name: pl, dtype: float64
-2733.718300000001


In [80]:
pd.to_numeric(all_transactions['halfSpreadCost']).sum()

4631.0979

In [82]:
orders = pd.read_csv('/work/data/orders/all_orders.csv')
orders['Time'] = pd.to_datetime(orders['Time'])
all_transactions['id'] = pd.to_numeric(all_transactions['id'])
all_transactions = pd.merge(all_transactions,orders,left_on='id', right_on = 'Order_ID',how='inner')

In [90]:
trans_orders = _dntk.execute_sql(
  'WITH trading_orders AS (\n    SELECT \n        CAST(all1.batchID AS INTEGER) as batchID,base.accountID,base.Strategy,\n        base.pl,\n        date_trunc(\'minute\',all1.time),\n        date_trunc(\'minute\', all1.time - INTERVAL \'6 minutes\') AS order_time\n    FROM all_transactions base \n    LEFT JOIN all_transactions order_filled \n        ON CAST(base.orderID AS INTEGER) = order_filled.id\n    LEFT JOIN all_transactions all1 \n        ON order_filled.batchID = all1.id\n    WHERE base.reason IN (\'TAKE_PROFIT_ORDER\', \'TRAILING_STOP_LOSS_ORDER\')\n)\nSELECT * FROM trading_orders\nORDER BY batchID ',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
trans_orders

,batchID,accountID,Strategy,pl,"date_trunc('minute', all1.""time"")",order_time


In [92]:
all_transactions['orderID'].unique()

array([nan], dtype=object)

In [94]:
all_transactions

,id,accountID,userID,batchID,requestID,time,type,divisionID,siteID,accountUserID,...,date,Time,Strategy,Type,Quantity,Price,Take_Profit,Stop_Loss,Order_ID,Account_ID
0,64,101-004-31059296-003,31059296,64,115401708806681736,2025-04-08 10:21:03.117912447+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 10:21:03.117912447+00:00,volume_based_hedge,BUY,1000000,1.09172,1.093358,0.0006,64,101-004-31059296-003
1,70,101-004-31059296-003,31059296,70,61358517052888566,2025-04-08 10:36:03.490659841+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 10:36:03.490659841+00:00,volume_based_hedge,BUY,1000000,1.09327,1.094910,0.0006,70,101-004-31059296-003
2,76,101-004-31059296-003,31059296,76,79372918077696599,2025-04-08 10:46:03.448784390+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 10:46:03.448784390+00:00,volume_based_hedge,BUY,1000000,1.09335,1.094990,0.0006,76,101-004-31059296-003
3,83,101-004-31059296-003,31059296,83,97387324142281340,2025-04-08 11:16:04.942459334+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 11:16:04.942459334+00:00,volume_based_hedge,BUY,1000000,1.09387,1.095511,0.0006,83,101-004-31059296-003
4,87,101-004-31059296-003,31059296,87,115401723913652606,2025-04-08 11:21:05.129298655+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 11:21:05.129298655+00:00,volume_based_hedge,BUY,1000000,1.09424,1.095881,0.0006,87,101-004-31059296-003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1214,101-004-31059296-001,31059296,1214,79373009941344415,2025-04-08 16:51:05.947268138+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 16:51:05.947268138+00:00,volume_based,BUY,1000000,1.09290,1.094539,0.0006,1214,101-004-31059296-001
70,1220,101-004-31059296-001,31059296,1220,97387409705552144,2025-04-08 16:56:04.836491276+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 16:56:04.836491276+00:00,volume_based,BUY,1000000,1.09250,1.094139,0.0006,1220,101-004-31059296-001
71,1228,101-004-31059296-001,31059296,1228,97387419778505592,2025-04-08 17:36:05.672488782+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 17:36:05.672488782+00:00,volume_based,SELL,-1000000,1.09416,1.092519,0.0006,1228,101-004-31059296-001
72,1232,101-004-31059296-001,31059296,1232,79373022523584459,2025-04-08 17:41:05.774516244+00:00,MARKET_ORDER,NaN,NaN,NaN,...,2025-04-08,2025-04-08 17:41:05.774516244+00:00,volume_based,SELL,-1000000,1.09307,1.091430,0.0006,1232,101-004-31059296-001


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>